This is for merging all the truth tables together

In [1]:
import os, sys, glob
import numpy as np

from astropy.io import fits
from astropy.table import Table, join, vstack
from astropy.io import fits

import pandas as pd
import fnmatch

import desispec.io

#sys.path.append("/global/homes/r/rtojeiro/SV/python")
import VI_utils as VI

import matplotlib.pyplot as plt 
import importlib
importlib.reload(VI)
pd.set_option('display.max_rows', 5)

In [2]:
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/andes/tiles/'
tiles = ['66003']
nights = ['20200315']
petals = ['0','1','2','3','4','5','6','7','8','9']

In [3]:
VI_dir = os.environ['HOME']+'/projects/VI_files/Andes_reinspection/BGS/'

In [4]:
vi = VI.read_individual_VI(VI_dir)

Reading all individual VI files in /global/homes/t/tlan/projects/VI_files/Andes_reinspection/BGS/
desi-vi_BGS_reinspection_70500_2_AMA.csv
desi-vi_BGS_reinspection_70500_2_MJO.csv
desi-vi_BGS_reinspection_70500_10_SMC.csv
desi-vi_BGS_reinspection_66003_3_SMC.csv
desi-vi_BGS_reinspection_70500_9_SYB.csv
desi-vi_BGS_reinspection_70500_3_CHH.csv
desi-vi_BGS_reinspection_66003_1_HSH.csv
desi-vi_BGS_reinspection_70500_9_AMA.csv
desi-vi_BGS_reinspection_70500_10_AMA.csv
desi-vi_BGS_reinspection_70500_10_CHH.csv
desi-vi_BGS_reinspection_66003_3_MJO.csv
desi-vi_BGS_reinspection_70500_3_MJO.csv
desi-vi_BGS_reinspection_70500_5_AMA.csv
desi-vi_BGS_reinspection_70500_6_SAM.csv
desi-vi_BGS_reinspection_70500_7_SYB.csv
desi-vi_BGS_reinspection_66003_2_MJO.csv
desi-vi_BGS_reinspection_66003_1_MJO.csv
desi-vi_BGS_reinspection_70500_7_AMA.csv
desi-vi_BGS_reinspection_66003_4_MJO.csv
desi-vi_BGS_reinspection_70500_5_MJO.csv
desi-vi_BGS_reinspection_70500_8_AMA.csv
desi-vi_BGS_reinspection_70500_2_CHH.c

In [5]:
vi

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment
0,35191196661843077,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.2732,CHH,1,C,NaN,NaN,NaN
1,35191196661843440,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.2080,CHH,3,NaN,NaN,NaN,Dn4000 + Continuum
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1824,35191220447743627,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.3579,MJO,4,NaN,NaN,NaN,(+ a QSO at z=2.1879?)
1825,35191220451937005,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.3305,MJO,4,NaN,NaN,NaN,H&K + 4000A


In [6]:
vi.loc[vi['VI_issue'].isnull(),'VI_issue']='--'
vi['all VI issue'] = vi.groupby('TARGETID')['VI_issue'].transform(lambda x: '|'.join(x))
unique_vi = vi.drop_duplicates(subset=['TARGETID'])

In [7]:
unique_vi
#vi.loc[vi['VI_comment'].isnull(),'VI_comment']='--'
#vi['all VI comments'] = vi.groupby('TARGETID')['VI_comment'].transform(lambda x: '|'.join(x))


,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,all VI issue
0,35191196661843077,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.2732,CHH,1,C,NaN,NaN,NaN,C|--|R
1,35191196661843440,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.2080,CHH,3,--,NaN,NaN,Dn4000 + Continuum,--|R|R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1385,35191220447743627,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.3579,AMA,4,--,NaN,NaN,NaN,--|--
1386,35191220451937005,52932,20200303,70500,0.34.4,-1,-1,GALAXY,0.3305,AMA,2,--,NaN,NaN,weak features and continuum,--|--


In [8]:
filenames = ['truth_table_Andes_reinspection_BGS_66003_20200315.csv', ]

truth_table = pd.read_csv(VI_dir+filenames[0])
for i in range(1,len(filenames)):
    partial_table = pd.read_csv(VI_dir+filenames[i])
    truth_table = truth_table.append(partial_table, ignore_index=True,sort=False)

In [9]:
truth_table

,TARGETID,Redrock_z,Redrock_spectype,best redshift,best spectype,vi_combined_flag,all VI comments,merger comment,N_VI,DELTACHI2,FIBER
0,35191325330507556,0.3928,GALAXY,0.3928,GALAXY,3.5,--|H&K Ha OII G MgI,none,2,100.725652,485
1,35191325330508207,0.4173,GALAXY,0.4173,GALAXY,4.0,--|H&K G MgI,none,2,465.061594,490
...,...,...,...,...,...,...,...,...,...,...,...
187,35191363104410851,0.1056,GALAXY,0.1056,GALAXY,4.0,--|--,none,2,139.500662,2941
188,35191366107529267,0.1993,GALAXY,0.1993,GALAXY,4.0,OII doublet resolved|--,none,2,3695.406035,2664


In [10]:
truth_table=truth_table.merge(unique_vi[['TARGETID', 'all VI issue']], on='TARGETID', how='left')


In [11]:
truth_table

,TARGETID,Redrock_z,Redrock_spectype,best redshift,best spectype,vi_combined_flag,all VI comments,merger comment,N_VI,DELTACHI2,FIBER,all VI issue
0,35191325330507556,0.3928,GALAXY,0.3928,GALAXY,3.5,--|H&K Ha OII G MgI,none,2,100.725652,485,--|--
1,35191325330508207,0.4173,GALAXY,0.4173,GALAXY,4.0,--|H&K G MgI,none,2,465.061594,490,--|--
...,...,...,...,...,...,...,...,...,...,...,...,...
187,35191363104410851,0.1056,GALAXY,0.1056,GALAXY,4.0,--|--,none,2,139.500662,2941,--|--
188,35191366107529267,0.1993,GALAXY,0.1993,GALAXY,4.0,OII doublet resolved|--,none,2,3695.406035,2664,--|--


In [12]:
truth_table = truth_table.rename(columns={"best redshift":"best z", "vi_combined_flag":"best quality"})


In [13]:
truth_table.keys()

Index(['TARGETID', 'Redrock_z', 'Redrock_spectype', 'best z', 'best spectype',
       'best quality', 'all VI comments', 'merger comment', 'N_VI',
       'DELTACHI2', 'FIBER', 'all VI issue'],
      dtype='object')

In [14]:
tf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
tspec = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')

for i in range(1,len(petals)):
    tn = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
    tnf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
    tspec = vstack([tspec,tn])
    tf = vstack([tf,tnf])

EXPID = list(set(tf['EXPID']))[0]
tf = tf[tf['EXPID']==EXPID]
tspec_df = tspec['TARGETID','ZWARN', 'ZERR'].to_pandas()
tf_df = tf['TARGETID','TARGET_RA','TARGET_DEC','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z','MW_TRANSMISSION_G', 'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z'].to_pandas()
print(len(tspec_df))
print(len(tf_df))

truth_table = truth_table.merge(tf_df, how='left', on='TARGETID')
truth_table = truth_table.merge(tspec_df, how='left', on='TARGETID')
#tspec_df = tspec['TARGETID','ZWARN', 'ZERR' ].to_pandas()
#tf_df = tf['TARGETID','MW_TRANSMISSION_G', 'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z'].to_pandas()

#tf_df = tf_df.rename(columns={"TARGETID": "TargetID"})
#tspec_df = tspec_df.rename(columns={"TARGETID": "TargetID"})

#truth_table = truth_table.merge(tf_df, how='left', on='TargetID')
#truth_table = truth_table.merge(tspec_df, how='left', on='TARGETID')

5000
5000


In [15]:
truth_table
#truth_table.columns

,TARGETID,Redrock_z,Redrock_spectype,best z,best spectype,best quality,all VI comments,merger comment,N_VI,DELTACHI2,...,FLUX_R,FLUX_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,ZWARN,ZERR
0,35191325330507556,0.3928,GALAXY,0.3928,GALAXY,3.5,--|H&K Ha OII G MgI,none,2,100.725652,...,7.630687,17.541019,0.614622,2.513090,5.776959,0.883664,0.920064,0.954468,0,0.000090
1,35191325330508207,0.4173,GALAXY,0.4173,GALAXY,4.0,--|H&K G MgI,none,2,465.061594,...,13.275474,30.919426,1.080219,5.557986,12.944905,0.918063,0.944040,0.968302,0,0.000053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,35191363104410851,0.1056,GALAXY,0.1056,GALAXY,4.0,--|--,none,2,139.500662,...,8.991776,13.379365,0.993658,1.872894,2.786783,0.948060,0.964709,0.980104,0,0.000020
188,35191366107529267,0.1993,GALAXY,0.1993,GALAXY,4.0,OII doublet resolved|--,none,2,3695.406035,...,9.644387,13.249196,2.841585,4.746860,6.521106,0.959249,0.972364,0.984446,0,0.000003


In [17]:
truth_table = truth_table[['TARGETID','TARGET_RA','TARGET_DEC','Redrock_z', 'Redrock_spectype', 'best z', 'best spectype',
       'best quality', 'N_VI', 'all VI issue', 'all VI comments', 'merger comment', 'DELTACHI2', 'ZWARN', 'ZERR','FIBER', 
        'FLUX_G', 'FLUX_R', 'FLUX_Z',
       'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'MW_TRANSMISSION_G',
       'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z']]

In [18]:
save_file = 'Truth_table_Andes_reinspection_BGS_66003_20200315_v1.csv'

In [19]:
truth_table.to_csv(VI_dir+save_file)
